In [41]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
data = pd.read_csv('loan_data.csv')

In [43]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [44]:
data.shape

(614, 13)

<h3>Data Preprocessing</h3>

In [45]:
data.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [46]:
data.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [47]:
data['Gender'].fillna(data['Gender'].mode()[0], inplace=True)
data['Married'].fillna(data['Married'].mode()[0], inplace=True)
data['Dependents'].fillna(data['Dependents'].mode()[0], inplace=True)
data['Self_Employed'].fillna(data['Self_Employed'].mode()[0], inplace=True)
data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mode()[0], inplace=True)
data['Credit_History'].fillna(data['Credit_History'].mode()[0], inplace=True)
data['LoanAmount'].fillna(data['LoanAmount'].mean(), inplace=True)

In [48]:
data['Gender'] = data['Gender'].map({'Male': 0, 'Female': 1})
data['Married'] = data['Married'].map({'No': 0, 'Yes': 1})
data['Dependents'] = data['Dependents'].map({'0': 0, '1': 1, '2': 2, '3+': 3})
data['Education'] = data['Education'].map({'Graduate': 1, 'Not Graduate': 0})
data['Self_Employed'] = data['Self_Employed'].map({'No': 0, 'Yes': 1})
data['Property_Area'] = data['Property_Area'].map({'Rural': 0, 'Semiurban': 1, 'Urban': 2})
data['Loan_Status'] = data['Loan_Status'].map({'N': 0, 'Y': 1})

In [49]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,0,5849,0.0,146.412162,360.0,1.0,2,1
1,LP001003,0,1,1,1,0,4583,1508.0,128.000000,360.0,1.0,0,0
2,LP001005,0,1,0,1,1,3000,0.0,66.000000,360.0,1.0,2,1
3,LP001006,0,1,0,0,0,2583,2358.0,120.000000,360.0,1.0,2,1
4,LP001008,0,0,0,1,0,6000,0.0,141.000000,360.0,1.0,2,1


In [50]:
#scale
for i in data.columns[1:]:
    data[i] = (data[i] - data[i].min()) / (data[i].max() - data[i].min())

In [51]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0.0,0.0,0.000000,1.0,0.0,0.070489,0.000000,0.198860,0.74359,1.0,1.0,1.0
1,LP001003,0.0,1.0,0.333333,1.0,0.0,0.054830,0.036192,0.172214,0.74359,1.0,0.0,0.0
2,LP001005,0.0,1.0,0.000000,1.0,1.0,0.035250,0.000000,0.082489,0.74359,1.0,1.0,1.0
3,LP001006,0.0,1.0,0.000000,0.0,0.0,0.030093,0.056592,0.160637,0.74359,1.0,1.0,1.0
4,LP001008,0.0,0.0,0.000000,1.0,0.0,0.072356,0.000000,0.191027,0.74359,1.0,1.0,1.0


<h3>Model Building</h3> 

In [52]:
data = data.drop('Loan_ID', axis=1)
x = data.drop('Loan_Status', axis=1)
y = data['Loan_Status']
x.shape, y.shape

((614, 11), (614,))

In [53]:
x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 stratify=data['Loan_Status'],random_state=10,test_size=0.2)

In [54]:
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((491, 11), (491,)), ((123, 11), (123,)))

In [55]:
import keras
import tensorflow as tf

In [56]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense 

In [89]:
input_neurons = x_train.shape[1]
input_neurons

11

In [90]:
output_neurons = 1

In [91]:
hidden_layers = 2
layer_1 = 10
layer_2 = 5

In [92]:
model = Sequential()
model.add(InputLayer(input_shape=(input_neurons,)))
model.add(Dense(units=layer_1, activation='relu'))
model.add(Dense(units=layer_2, activation='relu'))
model.add(Dense(units=output_neurons, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 10)                120       
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 6         
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________


In [97]:
model.compile(loss='binary_crossentropy',optimizer='SGD',metrics=['accuracy'])

In [98]:
model_train = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50)

Epoch 1/50
16/16 [==============================] - 1s 27ms/step - loss: 0.5092 - accuracy: 0.8056 - val_loss: 0.5216 - val_accuracy: 0.8130
Epoch 2/50
16/16 [==============================] - 0s 7ms/step - loss: 0.4890 - accuracy: 0.8290 - val_loss: 0.5214 - val_accuracy: 0.8130
Epoch 3/50
16/16 [==============================] - 0s 6ms/step - loss: 0.5071 - accuracy: 0.8139 - val_loss: 0.5210 - val_accuracy: 0.8130
Epoch 4/50
16/16 [==============================] - 0s 5ms/step - loss: 0.4921 - accuracy: 0.8236 - val_loss: 0.5208 - val_accuracy: 0.8130
Epoch 5/50
16/16 [==============================] - 0s 5ms/step - loss: 0.5013 - accuracy: 0.8199 - val_loss: 0.5205 - val_accuracy: 0.8130
Epoch 6/50
16/16 [==============================] - 0s 5ms/step - loss: 0.5253 - accuracy: 0.8015 - val_loss: 0.5200 - val_accuracy: 0.8130
Epoch 7/50
16/16 [==============================] - 0s 4ms/step - loss: 0.4963 - accuracy: 0.8153 - val_loss: 0.5196 - val_accuracy: 0.8130
Epoch 8/50
16/16 [=

In [99]:
prediction = model.predict_classes(x_test)

/Users/CaitlanKrasinski/miniconda3/envs/av/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [100]:
accuracy_score(y_test, prediction)

0.8130081300813008